In [1]:
# iPython Settings
%load_ext autoreload
%autoreload 2

import os

import copy
import numpy as np
from itertools import compress

import plotly as py
import plotly.graph_objs as go
from plotly import tools

import pandas as pd
pd.options.mode.chained_assignment = None

from ase.visualize import view

# My Modules ******************************************************************
from dft_job_automat.job_analysis import DFT_Jobs_Analysis
from dft_job_automat.job_types_classes.dft_methods import DFT_Methods
from colors.colors import color_list

# from density_of_states.dos import plot_pdos_dos
from dft_post_analysis.bands import plot_bands
from dft_post_analysis.dos import plot_pdos_dos

from dft_post_analysis.dos_bands_combined import plot_pdos_bands

In [2]:
research_dir = os.environ["norskov_research"]
data_pre_dir = research_dir + \
    "/04_comp_clusters/02_DATA/01_fe_graph_proj/bare_slabs/jobs_bin"

Jobs = DFT_Jobs_Analysis(
    update_job_state=False,
    job_type_class=None,
    load_dataframe=True,
    dataframe_dir=data_pre_dir,
    working_dir=".",
    )

df = Jobs.filter_early_revisions(Jobs.data_frame)

# Selecting Relevent Columns
df = df[
    [
        'job_type',
        'max_revision',
        'overlayer_pos',
        'path',
        'revision_number',
        'spinpol',
        'system',
        'elec_energy',
        'pdos_data',
        'bands_data',
        'init_atoms',
        ]
    ]

In [3]:
plotly_root_dir = "__temp__"
perc_keep_pdos=0.05
perc_keep_bands=0.3

# Bare Fe

In [4]:
def make_filter_list(len_data, percent_keep):
    """
    """
    #| - filter_list

#     len_data = len(pdos_data[0])

    filter_list = np.random.choice(
        [True, False],
        size=len_data,
        p=[percent_keep, 1. - percent_keep]
        )
    
    return(filter_list)
    #__|
    
def filter_pdos_data(pdos_data, percent_keep=0.4):
    """
    """
    #| - filter_pdos_data
    len_data = len(pdos_data[0])
    filter_list = make_filter_list(len_data, percent_keep)

    # --------------------------------------------------------------------------------------

    new_pdos_data = ()

    # **************************
    new_pdos_data += (np.array(list(compress(pdos_data[0], filter_list))),)

    # **************************
    shape_len = len(pdos_data[1].shape)
    if shape_len == 1:
        tuple_2 = np.array(list(compress(pdos_data[1], filter_list)))

    elif shape_len == 2:
        tuple_2 = [
            np.array(list(compress(pdos_data[1][0], filter_list))),
            np.array(list(compress(pdos_data[1][1], filter_list)))
            ]


    new_pdos_data += (tuple_2,)

    # **************************
    new_list = []
    for i_ind, atom_i in enumerate(pdos_data[2]):
        dict_i = {}
        for key, value in atom_i.items():

            series_list_new = []
            for j_ind, data_series_i in enumerate(value):
                tmp = np.array(list(compress(data_series_i, filter_list)))
                series_list_new.append(tmp)

            dict_i[key] = series_list_new

        new_list.append(dict_i)

    new_pdos_data += (new_list,)    

    
    return(new_pdos_data)
    #__|

def filter_bands_data(bands_data, percent_keep=0.6):
    """
    """
    #| - filter_bands_data
    len_data = len(bands_data[2])
    filter_list = make_filter_list(len_data, percent_keep)

    
    shape_len = len(bands_data[4].shape)
    
    if shape_len == 3:
        spinpol = True
    elif shape_len == 2:
        spinpol = False

    new_bands_data = ()

    new_bands_data += (bands_data[0],)
    new_bands_data += (bands_data[1],)
    new_bands_data += (np.array(list(compress(bands_data[2], filter_list))),)
    new_bands_data += (bands_data[3],)

#     spinpol = True
    if spinpol:
        new_data = []
        for spin in bands_data[4]:       
            spin_i_new = []
            for series_i in spin.T:
                tmp = np.array(list(compress(series_i, filter_list)))
                spin_i_new.append(tmp)

            spin_i_new = np.array(spin_i_new).T
            new_data.append(spin_i_new)
            
    else:
#         new_data = []
#         for spin in bands_data[4]:       
        spin_i_new = []
        for series_i in bands_data[4].T:
            tmp = np.array(list(compress(series_i, filter_list)))
            spin_i_new.append(tmp)

        new_data = np.array(spin_i_new).T
#         new_data.append(spin_i_new)
        
            
    new_bands_data += (np.array(new_data),)

    return(new_bands_data)
    #__|

In [6]:
plot_name = plotly_root_dir + "/dos/pl_pdos_bands_Fe_slab.html"
plot_title = "FCC Iron (111) spinpol: False"
pdos_filter_dict = {
    }

entry = df.loc[4]
pdos_data = entry.pdos_data
bands_data = entry.bands_data

pdos_data = filter_pdos_data(pdos_data, percent_keep=perc_keep_pdos)
bands_data = filter_bands_data(bands_data, perc_keep_bands)

In [7]:
energies, dos, pdos = pdos_data

In [8]:
# view(entry.init_atoms)
# Band Analysis
bands_data_out, bands_layout = plot_bands(bands_data)

# PDOS & DOS Analysis
dos_data_out, pdos_data_out, dos_layout = plot_pdos_dos(
    pdos_data,
    entry.init_atoms,
    filter_dict=pdos_filter_dict,
    group="band",
    e_range=[-1, 1],
    plot_title="N-doped Graphene Supported on FCC Fe(111)",
    )

# del dos_layout["xaxis"]["range"]

fig = plot_pdos_bands(
    pdos_data_out,
    dos_data_out,
    bands_data_out,
    dos_layout,
    bands_layout,
    plot_title=plot_title,
    )
py.plotly.iplot(fig, filename=plot_name)

KSDKFJDSJIFJSIDJFISD
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]

